In [1]:
from telegram import Update
from telegram.ext import (
    ApplicationBuilder, CommandHandler, MessageHandler,
    ConversationHandler, filters
)
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import nest_asyncio
import asyncio
from datetime import datetime

# ===========================
# CONFIG
# ===========================
BOT_TOKEN = "7983087073:AAEZfSy-rwr-9cazH-vZyTzXOa6fS83kDjY"
CREDENTIAL_FILE = "vaanidukaan_cred.json"
SPREADSHEET_NAME = "Impact_1"
BUSINESS_SHEET_NAME = "Business_Registration"
PRODUCT_SHEET_NAME = "Product_Details"
ORDER_SHEET_NAME = "Order_Details"

BASE_CATEGORIES = ["Papad", "Pickle", "Ghee", "Handicrafts"]

# Brand names in each language (for display)
BRAND_EN = "Vyapar Saathi by SHG"
BRAND_HI = "व्यापार साथी बाय SHG"
BRAND_MR = "व्यापार साथी बाय SHG"
BRAND_GU = "વ્યાપાર સાથી બાય SHG"

LANGUAGES = {
    "1": "en",
    "2": "hi",
    "3": "mr",
    "4": "gu"
}

# ===========================
# LANGUAGE TEXTS
# ===========================
TEXTS = {
    "choose_lang": {
        "en": f"Select your preferred language for {BRAND_EN}:\n"
              "1️⃣ English\n2️⃣ Hindi\n3️⃣ Marathi\n4️⃣ Gujarati",
        "hi": f"{BRAND_HI} के लिए अपनी भाषा चुनें:\n"
              "1️⃣ English\n2️⃣ Hindi\n3️⃣ Marathi\n4️⃣ Gujarati",
        "mr": f"{BRAND_MR} साठी आपली भाषा निवडा:\n"
              "1️⃣ English\n2️⃣ Hindi\n3️⃣ Marathi\n4️⃣ Gujarati",
        "gu": f"{BRAND_GU} માટે તમારી ભાષા પસંદ કરો:\n"
              "1️⃣ English\n2️⃣ Hindi\n3️⃣ Marathi\n4️⃣ Gujarati",
    },
    "welcome": {
        "en": f"👋 Welcome to {BRAND_EN} Business Console!\n\n"
              "1️⃣ Register My Business\n2️⃣ Login\n3️⃣ Help",
        "hi": f"👋 {BRAND_HI} बिज़नेस कंसोल में आपका स्वागत है!\n\n"
              "1️⃣ अपना व्यवसाय रजिस्टर करें\n2️⃣ लॉगिन\n3️⃣ मदद",
        "mr": f"👋 {BRAND_MR} व्यवसाय कन्सोलमध्ये आपले स्वागत आहे!\n\n"
              "1️⃣ व्यवसाय नोंदणी\n2️⃣ लॉगिन\n3️⃣ मदत",
        "gu": f"👋 {BRAND_GU} બિઝનેસ કન્સોલમાં આપનું સ્વાગત છે!\n\n"
              "1️⃣ બિઝનેસ રજીસ્ટર કરો\n2️⃣ લોગિન\n3️⃣ મદદ",
    },
    "enter_business_name": {
        "en": "📋 Enter Business Name (English Only):",
        "hi": "📋 व्यवसाय का नाम दर्ज करें (अंग्रेजी में):",
        "mr": "📋 व्यवसायाचे नाव टाका (फक्त इंग्लिशमध्ये):",
        "gu": "📋 બિઝનેસનું નામ દાખલ કરો (ફક્ત અંગ્રેજીમાં):",
    },
    "enter_phone": {  # used for login
        "en": "📱 Enter Registered Phone Number:",
        "hi": "📱 पंजीकृत मोबाइल नंबर दर्ज करें:",
        "mr": "📱 नोंदणीकृत फोन नंबर टाका:",
        "gu": "📱 રજીસ્ટર્ડ ફોન નંબર દાખલ કરો:",
    },
    "invalid_option": {
        "en": "❌ Invalid option.",
        "hi": "❌ गलत विकल्प।",
        "mr": "❌ चुकीचा पर्याय.",
        "gu": "❌ ખોટો વિકલ્પ.",
    },
    "owner_name_prompt": {
        "en": "👤 Owner Name (English Only):",
        "hi": "👤 मालिक का नाम (अंग्रेजी में):",
        "mr": "👤 मालकाचे नाव (इंग्लिशमध्ये):",
        "gu": "👤 માલિકનું નામ (ફક્ત અંગ્રેજીમાં):",
    },
    "phone_prompt_reg": {
        "en": "📞 Phone Number:",
        "hi": "📞 मोबाइल नंबर:",
        "mr": "📞 फोन नंबर:",
        "gu": "📞 ફોન નંબર:",
    },
    "location_prompt": {
        "en": "📍 Location (City, State in English):",
        "hi": "📍 स्थान (शहर, राज्य अंग्रेजी में):",
        "mr": "📍 ठिकाण (शहर, राज्य इंग्लिशमध्ये):",
        "gu": "📍 સ્થળ (શહેર, રાજ્ય અંગ્રેજીમાં):",
    },
    "upi_prompt": {
        "en": "💰 UPI ID (or type Skip):",
        "hi": "💰 UPI ID (या 'Skip' लिखें):",
        "mr": "💰 UPI ID (किंवा 'Skip' लिहा):",
        "gu": "💰 UPI ID (અથવા 'Skip' લખો):",
    },
    "support_text": {
        "en": "📞 support@vyaparsaathi.com",
        "hi": "📞 सहायता: support@vyaparsaathi.com",
        "mr": "📞 मदत: support@vyaparsaathi.com",
        "gu": "📞 મદદ: support@vyaparsaathi.com",
    },
    "invalid_menu_choice": {
        "en": "❌ Invalid option. Please use the shown numbers.",
        "hi": "❌ गलत विकल्प। कृपया दिए गए नंबर का उपयोग करें।",
        "mr": "❌ चुकीचा पर्याय. दिलेले क्रमांक वापरा.",
        "gu": "❌ ખોટો વિકલ્પ. બતાવેલા નંબરનો ઉપયોગ કરો.",
    },
    "add_product_name": {
        "en": "🆕 Enter Product Name (English Only):",
        "hi": "🆕 प्रोडक्ट का नाम (अंग्रेजी में):",
        "mr": "🆕 उत्पादनाचे नाव (इंग्लिशमध्ये):",
        "gu": "🆕 પ્રોડક્ટનું નામ (ફક્ત અંગ્રેજીમાં):",
    },
    "no_products": {
        "en": "📦 No products found. Add using 1️⃣ from menu.",
        "hi": "📦 कोई प्रोडक्ट नहीं मिला। मेनू से 1️⃣ द्वारा जोड़ें।",
        "mr": "📦 कोणतेही प्रोडक्ट नाहीत. मेनूमधून 1️⃣ वापरून जोडा.",
        "gu": "📦 કોઈ પ્રોડક્ટ નથી. મેનુમાંથી 1️⃣ દબાવીને ઉમેરો.",
    },
    "no_orders": {
        "en": "📭 No orders yet.",
        "hi": "📭 अभी तक कोई ऑर्डर नहीं।",
        "mr": "📭 अजून कोणतेही ऑर्डर नाहीत.",
        "gu": "📭 હજી સુધી કોઈ ઓર્ડર નથી.",
    },
    "logout_success": {
        "en": "👋 Logged Out Successfully.",
        "hi": "👋 लॉगआउट सफल रहा।",
        "mr": "👋 लॉगआउट यशस्वी.",
        "gu": "👋 સફળતાપૂર્વક લોગઆઉટ થયું.",
    },
    "wrong_phone_login": {
        "en": "❌ Wrong number. Try again:",
        "hi": "❌ गलत नंबर। दुबारा प्रयास करें:",
        "mr": "❌ चुकीचा नंबर. पुन्हा प्रयत्न करा:",
        "gu": "❌ ખોટો નંબર. ફરી પ્રયાસ કરો:",
    },
    "product_edit_prompt": {
        "en": "What do you want to edit?\n1 Name\n2 Category\n3 Price\n4 Weight\n5 Stock\n6 Image URL",
        "hi": "क्या बदलना चाहते हैं?\n1 नाम\n2 कैटेगरी\n3 कीमत\n4 वजन/साइज़\n5 स्टॉक\n6 इमेज URL",
        "mr": "काय बदलायचे आहे?\n1 नाव\n2 कॅटेगरी\n3 किंमत\n4 वजन/साइझ\n5 स्टॉक\n6 इमेज URL",
        "gu": "શું બદલવું છે?\n1 નામ\n2 કેટેગરી\n3 કિંમત\n4 વજન/સાઈઝ\n5 સ્ટોક\n6 ઇમેજ URL",
    },
    "enter_new_value": {
        "en": "Enter new value:",
        "hi": "नया मान दर्ज करें:",
        "mr": "नवीन मूल्य टाका:",
        "gu": "નવું મૂલ્ય દાખલ કરો:",
    },
    "updated_success": {
        "en": "✔ Updated! Type 'menu' to return.",
        "hi": "✔ अपडेट हो गया! वापस जाने के लिए 'menu' लिखें।",
        "mr": "✔ अपडेट झाले! परत जाण्यासाठी 'menu' टाइप करा.",
        "gu": "✔ અપડેટ થયું! પાછા જવા માટે 'menu' લખો.",
    },
    "delete_confirm": {
        "en": "Are you sure you want to delete? yes/no",
        "hi": "क्या आप सच में हटाना चाहते हैं? yes/no",
        "mr": "खरंच डिलीट करायचे? yes/no",
        "gu": "ખરેખર કાઢી નાંખવું છે? yes/no",
    },
    "deleted_success": {
        "en": "🗑 Deleted!",
        "hi": "🗑 हटा दिया गया!",
        "mr": "🗑 डिलीट झाले!",
        "gu": "🗑 કાઢી નાંખ્યું!",
    },
    "delete_cancel": {
        "en": "❌ Delete cancelled.",
        "hi": "❌ हटाना रद्द किया गया।",
        "mr": "❌ डिलीट रद्द केले.",
        "gu": "❌ ડિલીટ રદ્દ થયું.",
    },
    "cancel_generic": {
        "en": "Cancelled. Type 'menu' to go back.",
        "hi": "रद्द किया गया। वापस जाने के लिए 'menu' लिखें।",
        "mr": "रद्द केले. परत जाण्यासाठी 'menu' टाइप करा.",
        "gu": "રદ્દ થયું. પાછા જવા માટે 'menu' લખો.",
    },
}

def T(update, context, key):
    lang = context.user_data.get("lang", "en")
    return TEXTS.get(key, {}).get(lang, key)

# ===========================
# GOOGLE SHEET SETUP
# ===========================
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIAL_FILE, scope)
client = gspread.authorize(creds)

business_sheet = client.open(SPREADSHEET_NAME).worksheet(BUSINESS_SHEET_NAME)
product_sheet = client.open(SPREADSHEET_NAME).worksheet(PRODUCT_SHEET_NAME)
order_sheet = client.open(SPREADSHEET_NAME).worksheet(ORDER_SHEET_NAME)

# ===========================
# STATES
# ===========================
(
    SELECT_LANGUAGE,
    BUSINESS_NAME, OWNER_NAME, PHONE, LOCATION, UPI,
    LOGIN_PHONE,
    PROD_NAME, PROD_CATEGORY, PROD_PRICE, PROD_WEIGHT, PROD_STOCK, PROD_IMAGE,
    SELECT_ORDER, SELECT_STATUS,
    SELECT_PRODUCT_ACTION, EDIT_PRODUCT_FIELD, EDIT_PRODUCT_VALUE,
    CONFIRM_DELETE_PRODUCT,
    MAIN_MENU,
) = range(20)


# ===========================
# START COMMAND
# ===========================
# ===========================
# START COMMAND
# ===========================
async def start(update: Update, context):
    context.user_data.clear()
    context.user_data["logged_in"] = False

    # Send Business Bot Logo Image 👇
    try:
        with open("logo_busi_1.jpg", "rb") as img:
            await update.message.reply_photo(
                photo=img,
                caption="👋 Welcome to Vyapar Saathi Business Console"
            )
    except Exception as e:
        print("⚠️ Image send failed:", e)

    # Ask for language after sending image
    await update.message.reply_text(TEXTS["choose_lang"]["en"])
    return SELECT_LANGUAGE



# ===========================
# SELECT LANGUAGE
# ===========================
async def select_language(update: Update, context):
    lang_choice = update.message.text.strip()
    if lang_choice not in LANGUAGES:
        await update.message.reply_text(TEXTS["choose_lang"]["en"])
        return SELECT_LANGUAGE

    context.user_data["lang"] = LANGUAGES[lang_choice]
    return await show_main_menu(update, context)


# ===========================
# MAIN MENU UI
# ===========================
async def show_main_menu(update: Update, context):
    logged_in = context.user_data.get("logged_in", False)
    lang = context.user_data.get("lang", "en")

    if not logged_in:
        await update.message.reply_text(T(update, context, "welcome"))
    else:
        business_name = context.user_data.get("business_name")
        owner_name = context.user_data.get("owner_name")

        if lang == "en":
            text = (
                f"Welcome Back to {BRAND_EN}, {owner_name}! 👋\n"
                f"🏪 Store: {business_name}\n\n"
                "1️⃣ Add Product\n"
                "2️⃣ View / Edit / Delete Products\n"
                "3️⃣ View Orders\n"
                "4️⃣ Update Order Status\n"
                "5️⃣ Payments & Reports\n"
                "6️⃣ View Stock\n"
                "7️⃣ Help\n"
                "8️⃣ Logout\n\n"
                "💡 Type 'menu' anytime to return here."
            )
        elif lang == "hi":
            text = (
                f"{BRAND_HI} पर वापसी पर स्वागत है, {owner_name}! 👋\n"
                f"🏪 स्टोर: {business_name}\n\n"
                "1️⃣ प्रोडक्ट जोड़ें\n"
                "2️⃣ प्रोडक्ट देखें / एडिट / डिलीट\n"
                "3️⃣ ऑर्डर देखें\n"
                "4️⃣ ऑर्डर स्टेटस अपडेट\n"
                "5️⃣ पेमेंट्स और रिपोर्ट्स\n"
                "6️⃣ स्टॉक देखें\n"
                "7️⃣ मदद\n"
                "8️⃣ लॉगआउट\n\n"
                "💡 कभी भी 'menu' लिखकर यहाँ आएं।"
            )
        elif lang == "mr":
            text = (
                f"{BRAND_MR} मध्ये परत स्वागत आहे, {owner_name}! 👋\n"
                f"🏪 दुकान: {business_name}\n\n"
                "1️⃣ प्रोडक्ट जोडा\n"
                "2️⃣ प्रोडक्ट पाहा / एडिट / डिलीट\n"
                "3️⃣ ऑर्डर पाहा\n"
                "4️⃣ ऑर्डर स्टेटस अपडेट\n"
                "5️⃣ पेमेंट्स आणि रिपोर्ट्स\n"
                "6️⃣ स्टॉक पाहा\n"
                "7️⃣ मदत\n"
                "8️⃣ लॉगआउट\n\n"
                "💡 'menu' टाइप करून केव्हाही परत या."
            )
        else:  # gu
            text = (
                f"{BRAND_GU} પર પાછા આવી, {owner_name}! 👋\n"
                f"🏪 સ્ટોર: {business_name}\n\n"
                "1️⃣ પ્રોડક્ટ ઉમેરો\n"
                "2️⃣ પ્રોડક્ટ જુઓ / એડિટ / ડિલીટ\n"
                "3️⃣ ઓર્ડર જુઓ\n"
                "4️⃣ ઓર્ડર સ્ટેટસ અપડેટ\n"
                "5️⃣ પેમેન્ટ્સ અને રિપોર્ટ્સ\n"
                "6️⃣ સ્ટોક જુઓ\n"
                "7️⃣ મદદ\n"
                "8️⃣ લોગઆઉટ\n\n"
                "💡 ક્યારેય પણ 'menu' લખીને અહીં આવો."
            )

        await update.message.reply_text(text)

    return MAIN_MENU


# ===========================
# GLOBAL MENU HANDLER
# ===========================
async def go_to_main_menu(update: Update, context):
    """Clear current flow, keep login info, and go to dashboard."""
    logged_in = context.user_data.get("logged_in", False)
    business_id = context.user_data.get("business_id")
    business_name = context.user_data.get("business_name")
    owner_name = context.user_data.get("owner_name")
    lang = context.user_data.get("lang", "en")

    context.user_data.clear()
    if logged_in:
        context.user_data["logged_in"] = True
        context.user_data["business_id"] = business_id
        context.user_data["business_name"] = business_name
        context.user_data["owner_name"] = owner_name
    context.user_data["lang"] = lang

    return await show_main_menu(update, context)


# ===========================
# MENU HANDLER
# ===========================
async def handle_menu(update, context):
    text = update.message.text.strip()
    logged_in = context.user_data.get("logged_in", False)
    lang = context.user_data.get("lang", "en")

    if not logged_in:
        if text == "1":
            await update.message.reply_text(T(update, context, "enter_business_name"))
            return BUSINESS_NAME
        elif text == "2":
            await update.message.reply_text(T(update, context, "enter_phone"))
            return LOGIN_PHONE
        elif text == "3":
            await update.message.reply_text(T(update, context, "support_text"))
            return MAIN_MENU
        else:
            await update.message.reply_text(T(update, context, "invalid_menu_choice"))
            return MAIN_MENU

    else:
        if text == "1":
            await update.message.reply_text(T(update, context, "add_product_name"))
            return PROD_NAME
        elif text == "2":
            return await view_products(update, context)
        elif text == "3":
            return await view_orders(update, context)
        elif text == "4":
            return await update_order_start(update, context)
        elif text == "5":
            return await payment_reports(update, context)
        elif text == "6":
            return await view_stock(update, context)
        elif text == "7":
            await update.message.reply_text(T(update, context, "support_text"))
            return MAIN_MENU
        elif text == "8":
            context.user_data.clear()
            context.user_data["logged_in"] = False
            context.user_data["lang"] = lang
            await update.message.reply_text(T(update, context, "logout_success"))
            return MAIN_MENU
        else:
            await update.message.reply_text(T(update, context, "invalid_menu_choice"))
            return MAIN_MENU


# ===========================
# REGISTRATION FLOW
# ===========================
async def business_name(update, context):
    context.user_data["business_name"] = update.message.text.strip()
    await update.message.reply_text(T(update, context, "owner_name_prompt"))
    return OWNER_NAME


async def owner_name(update, context):
    context.user_data["owner_name"] = update.message.text.strip()
    await update.message.reply_text(T(update, context, "phone_prompt_reg"))
    return PHONE


async def phone(update, context):
    context.user_data["phone"] = update.message.text.strip()
    await update.message.reply_text(T(update, context, "location_prompt"))
    return LOCATION


async def location(update, context):
    context.user_data["location"] = update.message.text.strip()
    await update.message.reply_text(T(update, context, "upi_prompt"))
    return UPI


async def upi(update, context):
    lang = context.user_data.get("lang", "en")
    upi_val = update.message.text.strip()
    if upi_val.lower() == "skip":
        upi_val = ""
    context.user_data["upi"] = upi_val

    old = business_sheet.col_values(1)[1:]
    business_id = len(old) + 1

    business_sheet.append_row([
        business_id,
        context.user_data["business_name"],
        context.user_data["owner_name"],
        context.user_data["phone"],
        context.user_data["location"],
        context.user_data["upi"],
        "", "", "",
        datetime.now().strftime("%Y-%m-%d"),
        "Active"
    ])

    if lang == "en":
        msg = (
            f"🎉 Registered Successfully on {BRAND_EN}!\n"
            f"🆔 Business ID: {business_id}\n\n"
            "Now press 2️⃣ to Login from main menu."
        )
    elif lang == "hi":
        msg = (
            f"🎉 {BRAND_HI} पर रजिस्ट्रेशन सफल!\n"
            f"🆔 बिज़नेस ID: {business_id}\n\n"
            "अब मेनू से 2️⃣ दबाकर लॉगिन करें।"
        )
    elif lang == "mr":
        msg = (
            f"🎉 {BRAND_MR} वर नोंदणी यशस्वी!\n"
            f"🆔 व्यवसाय ID: {business_id}\n\n"
            "आता मेनूमधून 2️⃣ वापरून लॉगिन करा."
        )
    else:
        msg = (
            f"🎉 {BRAND_GU} પર રજીસ્ટ્રેશન સફળ!\n"
            f"🆔 બિઝનેસ ID: {business_id}\n\n"
            "હવે મેનુમાંથી 2️⃣ દબાવીને લોગિન કરો."
        )

    await update.message.reply_text(msg)
    return MAIN_MENU


# ===========================
# LOGIN FLOW
# ===========================
async def login_phone(update, context):
    phone = ''.join(filter(str.isdigit, update.message.text))[-10:]

    phones_raw = business_sheet.col_values(4)
    phones_norm = [''.join(filter(str.isdigit, p))[-10:] for p in phones_raw]

    if phone in phones_norm:
        idx = phones_norm.index(phone) + 1
        context.user_data["business_id"] = business_sheet.cell(idx, 1).value
        context.user_data["business_name"] = business_sheet.cell(idx, 2).value
        context.user_data["owner_name"] = business_sheet.cell(idx, 3).value
        context.user_data["logged_in"] = True
        return await show_main_menu(update, context)

    await update.message.reply_text(T(update, context, "wrong_phone_login"))
    return LOGIN_PHONE


# ===========================
# ADD PRODUCT FLOW (URL ONLY)
# ===========================
async def prod_name(update, context):
    context.user_data["prod_name"] = update.message.text.strip()

    cats = BASE_CATEGORIES
    lang = context.user_data.get("lang", "en")

    if lang == "en":
        msg = "📂 Choose Category:\n"
    elif lang == "hi":
        msg = "📂 कैटेगरी चुनें:\n"
    elif lang == "mr":
        msg = "📂 कॅटेगरी निवडा:\n"
    else:
        msg = "📂 કેટેગરી પસંદ કરો:\n"

    for i, c in enumerate(cats, start=1):
        msg += f"{i}️⃣ {c}\n"
    msg += f"{len(cats)+1}️⃣ Other\n"

    if lang == "en":
        msg += "Reply with number:"
    elif lang == "hi":
        msg += "नंबर लिखकर जवाब दें:"
    elif lang == "mr":
        msg += "क्रमांक टाइप करा:"
    else:
        msg += "નંબર લખીને જવાબ આપો:"

    await update.message.reply_text(msg)
    return PROD_CATEGORY


async def prod_category(update, context):
    text = update.message.text.strip()
    cats = BASE_CATEGORIES
    lang = context.user_data.get("lang", "en")

    if text.isdigit():
        choice = int(text)
        if choice == len(cats)+1:
            if lang == "en":
                msg = "✏ Enter Custom Category (English Only):"
            elif lang == "hi":
                msg = "✏ अपनी कैटेगरी का नाम लिखें (अंग्रेजी में):"
            elif lang == "mr":
                msg = "✏ स्वतःची कॅटेगरी (इंग्लिशमध्ये) टाका:"
            else:
                msg = "✏ તમારી જાતની કેટેગરી લખો (અંગ્રેજીમાં):"
            await update.message.reply_text(msg)
            context.user_data["await_cat"] = True
            return PROD_CATEGORY
        elif 1 <= choice <= len(cats):
            context.user_data["prod_category"] = cats[choice-1]
        else:
            if lang == "en":
                msg = "⚠ Invalid number."
            elif lang == "hi":
                msg = "⚠ गलत नंबर।"
            elif lang == "mr":
                msg = "⚠ चुकीचा क्रमांक."
            else:
                msg = "⚠ ખોટો નંબર."
            await update.message.reply_text(msg)
            return PROD_CATEGORY
    else:
        if context.user_data.get("await_cat"):
            context.user_data["prod_category"] = text
            context.user_data["await_cat"] = False
        else:
            if lang == "en":
                msg = "⚠ Invalid input. Please use numbers."
            elif lang == "hi":
                msg = "⚠ गलत इनपुट। कृपया नंबर का उपयोग करें।"
            elif lang == "mr":
                msg = "⚠ चुकीचा इनपुट. कृपया क्रमांक वापरा."
            else:
                msg = "⚠ ખોટો ઇનપુટ. કૃપા કરીને નંબરનો ઉપયોગ કરો."
            await update.message.reply_text(msg)
            return PROD_CATEGORY

    if lang == "en":
        msg = "💰 Enter Price:"
    elif lang == "hi":
        msg = "💰 प्रोडक्ट की कीमत दर्ज करें:"
    elif lang == "mr":
        msg = "💰 किंमत टाका:"
    else:
        msg = "💰 કિંમત દાખલ કરો:"
    await update.message.reply_text(msg)
    return PROD_PRICE


async def prod_price(update, context):
    lang = context.user_data.get("lang", "en")
    context.user_data["prod_price"] = update.message.text.strip()

    if lang == "en":
        msg = "⚖ Weight/Size (e.g., 200g):"
    elif lang == "hi":
        msg = "⚖ वजन/साइज़ (जैसे 200g):"
    elif lang == "mr":
        msg = "⚖ वजन/साइझ (उदा. 200g):"
    else:
        msg = "⚖ વજન/સાઈઝ (જેમ કે 200g):"
    await update.message.reply_text(msg)
    return PROD_WEIGHT


async def prod_weight(update, context):
    lang = context.user_data.get("lang", "en")
    context.user_data["prod_weight"] = update.message.text.strip()

    if lang == "en":
        msg = "📦 Stock Count:"
    elif lang == "hi":
        msg = "📦 स्टॉक की संख्या:"
    elif lang == "mr":
        msg = "📦 स्टॉक संख्या:"
    else:
        msg = "📦 સ્ટોકની સંખ્યા:"
    await update.message.reply_text(msg)
    return PROD_STOCK


async def prod_stock(update, context):
    lang = context.user_data.get("lang", "en")
    context.user_data["prod_stock"] = update.message.text.strip()

    if lang == "en":
        msg = (
            "🖼 Paste Public Image URL (.jpg/.png/.jpeg/.webp)\n"
            "Or type skip"
        )
    elif lang == "hi":
        msg = (
            "🖼 पब्लिक इमेज URL पेस्ट करें (.jpg/.png/.jpeg/.webp)\n"
            "या 'skip' लिखें"
        )
    elif lang == "mr":
        msg = (
            "🖼 सार्वजनिक इमेज URL पेस्ट करा (.jpg/.png/.jpeg/.webp)\n"
            "किंवा 'skip' लिहा"
        )
    else:
        msg = (
            "🖼 પબ્લિક ઇમેજ URL પેસ્ટ કરો (.jpg/.png/.jpeg/.webp)\n"
            "અથવા 'skip' લખો"
        )
    await update.message.reply_text(msg)
    return PROD_IMAGE


async def prod_image_text(update, context):
    lang = context.user_data.get("lang", "en")
    img = update.message.text.strip()
    if img.lower() == "skip":
        img = ""
    context.user_data["prod_image"] = img

    bid = context.user_data["business_id"]
    all_products = product_sheet.get_all_records()
    count = sum(1 for p in all_products if str(p["Business_ID"]) == str(bid))
    product_id = f"{bid}_pid{count + 1}"

    product_sheet.append_row([
        product_id,
        context.user_data["prod_name"],
        context.user_data["prod_category"],
        context.user_data["prod_price"],
        context.user_data["prod_weight"],
        context.user_data["prod_stock"],
        img,
        bid
    ])

    if lang == "en":
        msg = f"🎯 Product Added Successfully!\nID: {product_id}\nType 'menu' to go back."
    elif lang == "hi":
        msg = f"🎯 प्रोडक्ट सफलतापूर्वक जोड़ा गया!\nID: {product_id}\nवापस जाने के लिए 'menu' लिखें।"
    elif lang == "mr":
        msg = f"🎯 उत्पादन यशस्वीपणे जोडले गेले!\nID: {product_id}\nपरत जाण्यासाठी 'menu' टाइप करा."
    else:
        msg = f"🎯 પ્રોડક્ટ સફળતાપૂર્વક ઉમેરાયો!\nID: {product_id}\nપાછા જવા માટે 'menu' લખો."
    await update.message.reply_text(msg)
    return MAIN_MENU


# ===========================
# VIEW PRODUCTS / EDIT / DELETE
# ===========================
async def view_products(update, context):
    bid = context.user_data["business_id"]
    all_values = product_sheet.get_all_values()
    lang = context.user_data.get("lang", "en")

    if len(all_values) <= 1:
        await update.message.reply_text(T(update, context, "no_products"))
        return MAIN_MENU

    headers = all_values[0]
    values = all_values[1:]

    products = []
    for r, row in enumerate(values, start=2):
        data = dict(zip(headers, row))
        if str(data.get("Business_ID")) == str(bid):
            data["_row"] = r
            products.append(data)

    if not products:
        await update.message.reply_text(T(update, context, "no_products"))
        return MAIN_MENU

    context.user_data["product_list"] = products
    if lang == "en":
        msg = "🛒 Your Products:\n\n"
    elif lang == "hi":
        msg = "🛒 आपके प्रोडक्ट्स:\n\n"
    elif lang == "mr":
        msg = "🛒 तुमची उत्पादने:\n\n"
    else:
        msg = "🛒 તમારા પ્રોડક્ટ્સ:\n\n"

    for i, p in enumerate(products, start=1):
        msg += (
            f"{i}️⃣ {p['Product_Name']} | ₹{p['Price']} ({p['Weight_Size']})\n"
            f"Stock: {p['Stock']} | ID: {p['Product_ID']}\n\n"
        )

    if lang == "en":
        msg += "✏ Type: edit 2  |  🗑 delete 2\nType 'menu' to return."
    elif lang == "hi":
        msg += "✏ उदाहरण: edit 2  |  🗑 delete 2\nवापस जाने के लिए 'menu' लिखें।"
    elif lang == "mr":
        msg += "✏ उदा.: edit 2  |  🗑 delete 2\nपरत जाण्यासाठी 'menu' टाइप करा."
    else:
        msg += "✏ ઉદાહરણ: edit 2  |  🗑 delete 2\nપાછા જવા 'menu' લખો."

    await update.message.reply_text(msg)
    return SELECT_PRODUCT_ACTION


async def select_product_action(update, context):
    lang = context.user_data.get("lang", "en")
    text = update.message.text.split()
    if len(text) != 2:
        if lang == "en":
            msg = "Format: edit 2 OR delete 2"
        elif lang == "hi":
            msg = "फॉर्मेट: edit 2 या delete 2"
        elif lang == "mr":
            msg = "फॉरमॅट: edit 2 किंवा delete 2"
        else:
            msg = "ફોર્મેટ: edit 2 અથવા delete 2"
        await update.message.reply_text(msg)
        return SELECT_PRODUCT_ACTION

    action, idx_str = text
    if not idx_str.isdigit():
        await update.message.reply_text(T(update, context, "invalid_option"))
        return SELECT_PRODUCT_ACTION

    idx = int(idx_str)
    products = context.user_data["product_list"]
    if idx < 1 or idx > len(products):
        await update.message.reply_text(T(update, context, "invalid_option"))
        return SELECT_PRODUCT_ACTION

    selected = products[idx - 1]
    context.user_data["selected_product"] = selected

    if action == "edit":
        await update.message.reply_text(T(update, context, "product_edit_prompt"))
        return EDIT_PRODUCT_FIELD

    elif action == "delete":
        await update.message.reply_text(T(update, context, "delete_confirm"))
        return CONFIRM_DELETE_PRODUCT

    await update.message.reply_text(T(update, context, "invalid_option"))
    return SELECT_PRODUCT_ACTION


async def edit_product_field(update, context):
    choice = update.message.text.strip()
    fields = {
        "1": ("Product_Name", 2),
        "2": ("Category", 3),
        "3": ("Price", 4),
        "4": ("Weight_Size", 5),
        "5": ("Stock", 6),
        "6": ("Images", 7),
    }
    if choice not in fields:
        await update.message.reply_text(T(update, context, "invalid_option"))
        return EDIT_PRODUCT_FIELD

    field, col = fields[choice]
    context.user_data["edit_field"] = field
    context.user_data["edit_col"] = col
    await update.message.reply_text(T(update, context, "enter_new_value"))
    return EDIT_PRODUCT_VALUE


async def edit_product_value_text(update, context):
    new_val = update.message.text.strip()
    row = context.user_data["selected_product"]["_row"]
    col = context.user_data["edit_col"]
    product_sheet.update_cell(row, col, new_val)

    await update.message.reply_text(T(update, context, "updated_success"))
    return MAIN_MENU


async def confirm_delete_product(update, context):
    if update.message.text.lower() == "yes":
        row = context.user_data["selected_product"]["_row"]
        product_sheet.delete_rows(row)
        await update.message.reply_text(T(update, context, "deleted_success"))
    else:
        await update.message.reply_text(T(update, context, "delete_cancel"))
    return MAIN_MENU


# ===========================
# ORDERS / STATUS / REPORTS
# ===========================
async def view_orders(update, context):
    bid = context.user_data.get("business_id")
    orders = order_sheet.get_all_records()
    lang = context.user_data.get("lang", "en")

    business_orders = [o for o in orders if str(o["Business_ID"]) == str(bid)]
    if not business_orders:
        await update.message.reply_text(T(update, context, "no_orders"))
        return MAIN_MENU

    if lang == "en":
        msg = "📦 Orders:\n\n"
    elif lang == "hi":
        msg = "📦 ऑर्डर:\n\n"
    elif lang == "mr":
        msg = "📦 ऑर्डर्स:\n\n"
    else:
        msg = "📦 ઓર્ડર્સ:\n\n"

    for o in business_orders:
        msg += (
            f"📌 {o['Order_ID']}\n"
            f"👤 {o['Customer_Name']}\n"
            f"🛍️ {o['Product_Ordered']} x{o['Qunatity']}\n"
            f"🚚 {o['Order_status']}\n\n"
        )

    await update.message.reply_text(msg)
    return MAIN_MENU


async def update_order_start(update, context):
    bid = context.user_data["business_id"]
    orders = order_sheet.get_all_records()
    lang = context.user_data.get("lang", "en")
    business_orders = [o for o in orders if str(o["Business_ID"]) == str(bid)]

    if not business_orders:
        await update.message.reply_text(T(update, context, "no_orders"))
        return MAIN_MENU

    context.user_data["orders_list"] = business_orders

    if lang == "en":
        msg = "Select order:\n"
    elif lang == "hi":
        msg = "ऑर्डर चुनें:\n"
    elif lang == "mr":
        msg = "ऑर्डर निवडा:\n"
    else:
        msg = "ઓર્ડર પસંદ કરો:\n"

    for i, o in enumerate(business_orders, start=1):
        msg += f"{i}️⃣ {o['Order_ID']} - {o['Order_status']}\n"

    await update.message.reply_text(msg)
    return SELECT_ORDER


async def choose_order(update, context):
    lang = context.user_data.get("lang", "en")
    idx = update.message.text.strip()
    if not idx.isdigit():
        if lang == "en":
            msg = "Enter order number:"
        elif lang == "hi":
            msg = "ऑर्डर नंबर दर्ज करें:"
        elif lang == "mr":
            msg = "ऑर्डर नंबर टाका:"
        else:
            msg = "ઓર્ડર નંબર દાખલ કરો:"
        await update.message.reply_text(msg)
        return SELECT_ORDER

    idx = int(idx)
    orders = context.user_data["orders_list"]
    if idx < 1 or idx > len(orders):
        await update.message.reply_text(T(update, context, "invalid_option"))
        return SELECT_ORDER

    context.user_data["selected_order"] = orders[idx - 1]

    if lang == "en":
        msg = "Select new status:\n1 Pending\n2 Shipped\n3 Delivered"
    elif lang == "hi":
        msg = "नया स्टेटस चुनें:\n1 Pending\n2 Shipped\n3 Delivered"
    elif lang == "mr":
        msg = "नवीन स्टेटस निवडा:\n1 Pending\n2 Shipped\n3 Delivered"
    else:
        msg = "નવો સ્ટેટસ પસંદ કરો:\n1 Pending\n2 Shipped\n3 Delivered"

    await update.message.reply_text(msg)
    return SELECT_STATUS


async def set_status(update, context):
    status_map = {"1": "Pending", "2": "Shipped", "3": "Delivered"}
    choice = update.message.text.strip()
    lang = context.user_data.get("lang", "en")

    if choice not in status_map:
        await update.message.reply_text(T(update, context, "invalid_option"))
        return SELECT_STATUS

    new_status = status_map[choice]
    order = context.user_data["selected_order"]
    order_id = order["Order_ID"]
    ids = order_sheet.col_values(1)
    row = ids.index(order_id) + 1
    order_sheet.update_cell(row, 9, new_status)

    if lang == "en":
        msg = "✔ Status Updated."
    elif lang == "hi":
        msg = "✔ स्टेटस अपडेट हो गया।"
    elif lang == "mr":
        msg = "✔ स्टेटस अपडेट झाला."
    else:
        msg = "✔ સ્ટેટસ અપડેટ થયો."

    await update.message.reply_text(msg)
    return MAIN_MENU


async def payment_reports(update, context):
    bid = context.user_data["business_id"]
    orders = order_sheet.get_all_records()
    business_orders = [o for o in orders if str(o["Business_ID"]) == str(bid)]
    lang = context.user_data.get("lang", "en")

    if not business_orders:
        await update.message.reply_text(T(update, context, "no_orders"))
        return MAIN_MENU

    revenue = sum(int(o["Amount"]) for o in business_orders)
    total = len(business_orders)
    pending = sum(1 for o in business_orders if o["Order_status"] == "Pending")
    shipped = sum(1 for o in business_orders if o["Order_status"] == "Shipped")
    delivered = sum(1 for o in business_orders if o["Order_status"] == "Delivered")

    if lang == "en":
        msg = (
            "📊 Sales Report\n"
            f"Total Orders: {total}\n"
            f"Pending: {pending}\n"
            f"Shipped: {shipped}\n"
            f"Delivered: {delivered}\n"
            f"Revenue: ₹{revenue}"
        )
    elif lang == "hi":
        msg = (
            "📊 सेल्स रिपोर्ट\n"
            f"कुल ऑर्डर: {total}\n"
            f"Pending: {pending}\n"
            f"Shipped: {shipped}\n"
            f"Delivered: {delivered}\n"
            f"कुल राजस्व: ₹{revenue}"
        )
    elif lang == "mr":
        msg = (
            "📊 सेल्स रिपोर्ट\n"
            f"एकूण ऑर्डर्स: {total}\n"
            f"Pending: {pending}\n"
            f"Shipped: {shipped}\n"
            f"Delivered: {delivered}\n"
            f"उत्पन्न: ₹{revenue}"
        )
    else:
        msg = (
            "📊 સેલ્સ રિપોર્ટ\n"
            f"કુલ ઓર્ડર: {total}\n"
            f"Pending: {pending}\n"
            f"Shipped: {shipped}\n"
            f"Delivered: {delivered}\n"
            f"કુલ આવક: ₹{revenue}"
        )

    await update.message.reply_text(msg)
    return MAIN_MENU


async def view_stock(update, context):
    bid = context.user_data["business_id"]
    products = product_sheet.get_all_records()
    my_products = [p for p in products if str(p["Business_ID"]) == str(bid)]
    lang = context.user_data.get("lang", "en")

    if not my_products:
        await update.message.reply_text(T(update, context, "no_products"))
        return MAIN_MENU

    if lang == "en":
        msg = "📦 Stock Overview:\n\n"
    elif lang == "hi":
        msg = "📦 स्टॉक विवरण:\n\n"
    elif lang == "mr":
        msg = "📦 स्टॉक ओव्हरव्ह्यू:\n\n"
    else:
        msg = "📦 સ્ટોક ઓવરવ્યુ:\n\n"

    for p in my_products:
        msg += f"🛍️ {p['Product_Name']} — {p['Stock']} left\n"

    await update.message.reply_text(msg)
    return MAIN_MENU


# ===========================
# CANCEL
# ===========================
async def cancel(update, context):
    await update.message.reply_text(T(update, context, "cancel_generic"))
    return MAIN_MENU


# ===========================
# BOT HANDLERS
# ===========================
menu_filter = filters.Regex("(?i)^menu$")

conv_handler = ConversationHandler(
    entry_points=[CommandHandler("start", start)],
    states={
        SELECT_LANGUAGE: [
            MessageHandler(filters.TEXT & (~filters.COMMAND), select_language),
        ],
        MAIN_MENU: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), handle_menu),
        ],
        BUSINESS_NAME: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), business_name),
        ],
        OWNER_NAME: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), owner_name),
        ],
        PHONE: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), phone),
        ],
        LOCATION: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), location),
        ],
        UPI: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), upi),
        ],
        LOGIN_PHONE: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), login_phone),
        ],
        PROD_NAME: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), prod_name),
        ],
        PROD_CATEGORY: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), prod_category),
        ],
        PROD_PRICE: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), prod_price),
        ],
        PROD_WEIGHT: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), prod_weight),
        ],
        PROD_STOCK: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), prod_stock),
        ],
        PROD_IMAGE: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), prod_image_text),
        ],
        SELECT_PRODUCT_ACTION: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), select_product_action),
        ],
        EDIT_PRODUCT_FIELD: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), edit_product_field),
        ],
        EDIT_PRODUCT_VALUE: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), edit_product_value_text),
        ],
        CONFIRM_DELETE_PRODUCT: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), confirm_delete_product),
        ],
        SELECT_ORDER: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), choose_order),
        ],
        SELECT_STATUS: [
            MessageHandler(menu_filter, go_to_main_menu),
            MessageHandler(filters.TEXT & (~filters.COMMAND), set_status),
        ],
    },
    fallbacks=[CommandHandler("cancel", cancel)],
    allow_reentry=True
)

app = ApplicationBuilder().token(BOT_TOKEN).build()
app.add_handler(conv_handler)

nest_asyncio.apply()
print("🚀 Business Bot Running…")
asyncio.get_event_loop().run_until_complete(app.run_polling())


🚀 Business Bot Running…


RuntimeError: Cannot close a running event loop